In [1]:
%load_ext autoreload
%autoreload 2

import os
import shutil

import sinara_firmware_compiler as sinara

# Actual system description files that we want to use:

### `crate_df` contains every crate of Sinara hardware that we have physically assembled.

If you want the same crate twice but with different configuration values, then you have to make 2 crates with different names.

In [2]:
import pandas as pd

crate_df = sinara.make_crate_df([
    # The columns are:
    # unique crate name, either "kasli" or "kasli_soc", list of peripherals
    ["empty", "kasli_soc", []],
    ["mqva_dio", "kasli_soc", [sinara.dio_sma(ports=[port]) for port in range(0,12,1)]],
    ["mqva_ao", "kasli_soc", [sinara.fastino(ports=[port]) for port in range(0,4,2)]],
    ["mqva_dds", "kasli_soc", [sinara.urukul(ports=[port,port+1], dds="ad9910") for port in range(0,12,2)]],
    ["mqva_dds2", "kasli_soc", [sinara.urukul(ports=[port,port+1], dds="ad9910") for port in range(0,10,2)]\
                              +[sinara.urukul(ports=[10,11], dds="ad9912")]],
])
crate_df

,target,peripherals
crate_id,,
empty,kasli_soc,[]
mqva_dio,kasli_soc,[<sinara_firmware_compiler.sinara_hardware.dio...
mqva_ao,kasli_soc,[<sinara_firmware_compiler.sinara_hardware.fas...
mqva_dds,kasli_soc,[<sinara_firmware_compiler.sinara_hardware.uru...
mqva_dds2,kasli_soc,[<sinara_firmware_compiler.sinara_hardware.uru...


### `system_df` contains every combination of Sinara crates that we want to run together as a system.

If you want the same system twice but with different configuration values, then you have to make 2 systems with different names.

In [3]:
schema_keys = ["sed_lanes", "enable_wrpll", "core_addr"]
default_config = [32, True, "10.0.0.50"]
systems = [
    # The columns are:
    # unique system name, list of crate names (default config) or list of {crate name : custom_config}
    # The first crate is the master (or standalone), any following crates are satellites.
    # ["empty_standalone", ["empty"]],
    # ["empty_with_satellite", ["empty", "empty"]],
    # ["empty_with_custom_config", [{"empty" : [8, False, "ABCDE"]},
    #                               "empty"]],
    ["test_system", ["mqva_dds2"]],
    ["mqva_lab", ["mqva_dio", "mqva_ao", "mqva_dds"]],
    ["office_test_setup", ["mqva_dio", "mqva_ao", "mqva_dds2"]],
    ["mqva_lab_new", ["mqva_dio", "mqva_ao", "mqva_dds", "mqva_dds2"]],
]
artiq_zynq_branches_and_commit_hashes = {
    "master" : ["63157588bb8ed214bc9f24ade2d09b4ddefada03"],
}
check_commits_in_branches = True
artiq_zynq_path = "/opt/m1/workspace/artiq-zynq-original-untouched"
output_folder_path = os.path.realpath("system_description_files", strict=False)

system_df, compile_df = sinara.make_system_df_compile_df(
    crate_df, systems, schema_keys, default_config, artiq_zynq_branches_and_commit_hashes,
    check_commits_in_branches, artiq_zynq_path, output_folder_path
)
sinara.style(system_df, 120)

### `compile_df` is calculated automatically from `crate_df` and `system_df` and contains every variant that needs to be compiled.

In [4]:
sinara.style(compile_df, 150)

,target_instance,json_folder_path,json_file_path
variant,,,
crate_mqva_ao_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_fastino_2_addr_10.0.0.50,<sinara_firmware_compiler.sinara_hardware.kasli_soc object at 0x79d3e55d4d50>,/opt/m1/workspace/sinara-firmware-compiler/examples/system_description_files/crate_mqva_ao_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_fastino_2_addr_10.0.0.50,/opt/m1/workspace/sinara-firmware-compiler/examples/system_description_files/crate_mqva_ao_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_fastino_2_addr_10.0.0.50/crate_mqva_ao_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_fastino_2_addr_10.0.0.50.json
crate_mqva_dds_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50,<sinara_firmware_compiler.sinara_hardware.kasli_soc object at 0x79d3e55d48d0>,/opt/m1/workspace/sinara-firmware-compiler/examples/system_description_files/crate_mqva_dds_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50,/opt/m1/workspace/sinara-firmware-compiler/examples/system_description_files/crate_mqva_dds_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50/crate_mqva_dds_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50.json
crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50,<sinara_firmware_compiler.sinara_hardware.kasli_soc object at 0x79d3e55be710>,/opt/m1/workspace/sinara-firmware-compiler/examples/system_description_files/crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50,/opt/m1/workspace/sinara-firmware-compiler/examples/system_description_files/crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50/crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50.json
crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_standalone_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50,<sinara_firmware_compiler.sinara_hardware.kasli_soc object at 0x79d3e55d53d0>,/opt/m1/workspace/sinara-firmware-compiler/examples/system_description_files/crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_standalone_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50,/opt/m1/workspace/sinara-firmware-compiler/examples/system_description_files/crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_standalone_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50/crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_standalone_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50.json
crate_mqva_dio_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_master_wrpll_True_sed_32_periph_dio_sma_12_addr_10.0.0.50,<sinara_firmware_compiler.sinara_hardware.kasli_soc object at 0x79d3e55d5690>,/opt/m1/workspace/sinara-firmware-compiler/examples/system_description_files/crate_mqva_dio_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_master_wrpll_True_sed_32_periph_dio_sma_12_addr_10.0.0.50,/opt/m1/workspace/sinara-firmware-compiler/examples/sys

### Write system description files to disc based on `compile_df`.

`to_compile_file` lists the path of every system description file. If you made your own system description files manually but you want to compile the firmware for all of them automatically with `sinara-firmware-compiler`, then create `to_compile_file` manually and go to the next notebook.

In [5]:
os.makedirs(output_folder_path)
to_compile_file = os.path.join(output_folder_path, "to_compile.txt")

sinara.make_to_compile(to_compile_file, compile_df)

crate_mqva_ao_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_fastino_2_addr_10.0.0.50.json
crate_mqva_dds_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50.json
crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_satellite_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50.json
crate_mqva_dds2_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_standalone_wrpll_True_sed_32_periph_urukul_6_addr_10.0.0.50.json
crate_mqva_dio_kasli_soc_v1.1_artiq_zynq_master_63157588bb8ed214bc9f24ade2d09b4ddefada03_drtio_master_wrpll_True_sed_32_periph_dio_sma_12_addr_10.0.0.50.json


### Prepare for creating `device_db.py`, `dev_map.bin` and `dev_map.txt`.

`to_device_db_file` lists all Sinara systems. For each system, the paths of all system description files are listed in the correct drtio order. If you made your own system description files manually but you want to create `device_db.py`, `dev_map.bin` and `dev_map.txt` for all Sinara systems automatically with `sinara-firmware-compiler`, then create `to_device_db_file` manually and go to the next notebook.

In [6]:
to_device_db_file = os.path.join(output_folder_path, "to_device_db.txt")
artiq_branch = "master"
artiq_commit = "c50f3a1c12979717d045fccee8109e9d229abf3c"

device_db_df = sinara.make_to_device_db(
    to_device_db_file, system_df, compile_df, artiq_branch, artiq_commit
)
device_db_df

,artiq_branch,artiq_commit,output_file_path,master_json_file_path,satellite_0_json_file_path,satellite_1_json_file_path,satellite_2_json_file_path
0,master,c50f3a1c12979717d045fccee8109e9d229abf3c,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,None
1,master,c50f3a1c12979717d045fccee8109e9d229abf3c,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...
2,master,c50f3a1c12979717d045fccee8109e9d229abf3c,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,None
3,master,c50f3a1c12979717d045fccee8109e9d229abf3c,/opt/m1/workspace/sinara-firmware-compiler/exa...,/opt/m1/workspace/sinara-firmware-compiler/exa...,None,None,None


In [7]:
# shutil.rmtree(output_folder_path)